In [22]:
#Importing necessary libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [23]:
#Load in the data set
fifaDB = pd.read_csv("data.csv")

In [24]:
#Preview Data base
fifaDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
Unnamed: 0                  18207 non-null int64
ID                          18207 non-null int64
Name                        18207 non-null object
Age                         18207 non-null int64
Photo                       18207 non-null object
Nationality                 18207 non-null object
Flag                        18207 non-null object
Overall                     18207 non-null int64
Potential                   18207 non-null int64
Club                        17966 non-null object
Club Logo                   18207 non-null object
Value                       18207 non-null object
Wage                        18207 non-null object
Special                     18207 non-null int64
Preferred Foot              18159 non-null object
International Reputation    18159 non-null float64
Weak Foot                   18159 non-null float64
Skill Moves                 18159 non-null fl

In [25]:
fifaDB.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [26]:
#Drop unnecessary columns
Fifa = fifaDB.drop(['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Unnamed: 0', 'Photo', 'Flag', 'Club Logo', 'Joined', 'Loaned From', 'Contract Valid Until', 'Special', 'International Reputation', 'Real Face', "Nationality", "Age", "ID", "Release Clause", "Club","GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes" ], axis=1)

In [27]:
#Look for null values
Fifa.isna().sum() 

Name                0
Overall             0
Potential           0
Value               0
Wage                0
Preferred Foot     48
Weak Foot          48
Skill Moves        48
Work Rate          48
Body Type          48
Position           60
Jersey Number      60
Height             48
Weight             48
Crossing           48
Finishing          48
HeadingAccuracy    48
ShortPassing       48
Volleys            48
Dribbling          48
Curve              48
FKAccuracy         48
LongPassing        48
BallControl        48
Acceleration       48
SprintSpeed        48
Agility            48
Reactions          48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
Strength           48
LongShots          48
Aggression         48
Interceptions      48
Positioning        48
Vision             48
Penalties          48
Composure          48
Marking            48
StandingTackle     48
SlidingTackle      48
dtype: int64

In [28]:
#Drop null values
Fifa = Fifa.dropna()

In [29]:
list_wages = Fifa['Wage']
wages = []
for i in list_wages:
    hoi = i.strip('K').strip('€')
    wage = int(hoi) * 1000
    wages.append(wage)
    
Fifa['Wage'] = wages



In [30]:
list_lengths = np.array(Fifa['Height'])
empty_list = []
for x in list_lengths:
    splitted_values = str(x).split("'")
    feet = (float(x[0])) + (float(x[2])/10)
    centimeters = feet * 30.48
    empty_list.append(round(centimeters))

Fifa['Height'] = empty_list


In [31]:
list_value = Fifa['Value']
list_new_values = []
rows = list_value[:]

for v in rows:
    if v[-1] == 'M':
        strip_value = v.strip('M€')
        if len(strip_value) >= 4:
            yes = (strip_value[:3])
            new_value = float(yes) * 1000000
            list_new_values.append(int(new_value))
        else:
            new_value = float(strip_value) * 1000000
            list_new_values.append(int(new_value))
            
    else:
        strip_value = v.strip('K€')
        new_value = float(strip_value) * 1000
        
        list_new_values.append(int(new_value))

        
Fifa['Value'] = list_new_values

In [32]:
list_weight = Fifa['Weight']
weights = []
for i in list_weight:
    lbs_to_kg = i.strip('lbs')
    convert_weight = float(lbs_to_kg)/2.2046
    roundup_weight = round(convert_weight)
    weights.append(roundup_weight)

Fifa['Weight'] = weights

In [53]:
#Label encoder for work rate
le = preprocessing.LabelEncoder()
Fifa['Work Rate'] = Fifa[['Work Rate']].apply(le.fit_transform)
Fifa['Body Type'] = Fifa[['Body Type']].apply(le.fit_transform)
Fifa['Position'] = Fifa[['Position']].apply(le.fit_transform)

In [54]:
#onehot encoding
Fifa["Preferred Foot"] = pd.get_dummies(Fifa["Preferred Foot"])

In [55]:
#Remove Goalkeepers
Fifa = Fifa[Fifa.Position != 5]
Fifa.head(10)

,Name,Overall,Potential,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Work Rate,Body Type,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,L. Messi,94,94,110000000,565000,0,4.0,4.0,8,4,...,1.000000,0.510638,0.234043,1.000000,1.000000,75.0,1.021277,0.351064,0.297872,0.276596
1,Cristiano Ronaldo,94,94,77000000,405000,1,4.0,5.0,1,1,...,0.989362,0.670213,0.308511,1.010638,0.872340,85.0,1.010638,0.297872,0.329787,0.244681
2,Neymar Jr,92,93,118000000,290000,1,5.0,5.0,2,5,...,0.891304,0.608696,0.391304,0.967391,0.945652,81.0,1.021739,0.293478,0.260870,0.358696
4,K. De Bruyne,91,92,102000000,355000,1,5.0,4.0,0,6,...,1.000000,0.835165,0.670330,0.956044,1.032967,79.0,0.967033,0.747253,0.637363,0.560440
5,E. Hazard,91,91,93000000,340000,1,4.0,4.0,2,6,...,0.879121,0.593407,0.450549,0.956044,0.978022,86.0,1.000000,0.373626,0.296703,0.241758
6,L. Modrić,91,91,67000000,420000,1,4.0,4.0,0,3,...,0.901099,0.681319,0.912088,0.868132,1.010989,82.0,0.923077,0.659341,0.835165,0.802198
7,L. Suárez,91,91,80000000,455000,1,4.0,3.0,2,6,...,0.934066,0.956044,0.450549,1.010989,0.923077,85.0,0.934066,0.681319,0.494505,0.417582
8,Sergio Ramos,91,91,51000000,380000,1,3.0,3.0,2,6,...,0.648352,0.967033,0.989011,0.659341,0.692308,75.0,0.901099,0.956044,1.010989,1.000000
10,R. Lewandowski,90,90,77000000,205000,1,4.0,4.0,2,6,...,0.933333,0.888889,0.433333,1.011111,0.855556,88.0,0.955556,0.377778,0.466667,0.211111
11,T. Kroos,90,90,76000000,355000,1,5.0,3.0,8,6,...,1.022222,0.666667,0.911111,0.877778,0.955556,73.0,0.944444,0.800000,0.877778,0.766667


In [56]:
FifaSkills = Fifa[['Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle']]

In [57]:
FifaNormSkill = FifaSkills.div(Fifa.Overall, axis=0)
Fifa['Finishing'] = FifaNormSkill['Finishing']
Fifa['HeadingAccuracy'] = FifaNormSkill['HeadingAccuracy']
Fifa['ShortPassing'] = FifaNormSkill['ShortPassing']
Fifa['Volleys'] = FifaNormSkill['Volleys']
Fifa['Dribbling'] = FifaNormSkill['Dribbling']
Fifa['Curve'] = FifaNormSkill['Curve']
Fifa['FKAccuracy'] = FifaNormSkill['FKAccuracy']
Fifa['LongPassing'] = FifaNormSkill['LongPassing']
Fifa['BallControl'] = FifaNormSkill['BallControl']
Fifa['Acceleration'] = FifaNormSkill['Acceleration']
Fifa['SprintSpeed'] = FifaNormSkill['SprintSpeed']
Fifa['Agility'] = FifaNormSkill['Agility']
Fifa['Reactions'] = FifaNormSkill['Reactions']
Fifa['Balance'] = FifaNormSkill['Balance']
Fifa['ShotPower'] = FifaNormSkill['ShotPower']
Fifa['Jumping'] = FifaNormSkill['Jumping']
Fifa['Stamina'] = FifaNormSkill['Stamina']
Fifa['Strength'] = FifaNormSkill['Strength']
Fifa['LongShots'] = FifaNormSkill['LongShots']
Fifa['Aggression'] = FifaNormSkill['Aggression']
Fifa['Interceptions'] = FifaNormSkill['Interceptions']
Fifa['Positioning'] = FifaNormSkill['Positioning']
Fifa['Vision'] = FifaNormSkill['Vision']
Fifa['Composure'] = FifaNormSkill['Composure']
Fifa['Marking'] = FifaNormSkill['Marking']
Fifa['StandingTackle'] = FifaNormSkill['StandingTackle']
Fifa['SlidingTackle'] = FifaNormSkill['SlidingTackle']

c:\users\stane\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\stane\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\stane\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is tryin

In [58]:
X = Fifa
y = Fifa['Position']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [60]:
def savetocsv(dataframe, name):
    dataframe.to_csv(name, index = None, header=True)

In [61]:
savetocsv(X_train, "TrainFifa.csv")
savetocsv(X_val, "ValFifa.csv")
savetocsv(X_test, "TestFifa.csv")
savetocsv(y_train, "ytrainFifa.csv")
savetocsv(y_val, "yvalFifa.csv")
savetocsv(y_test, "ytestFifa.csv")